
### Submission Instructions:

Make sure you fill in any place that says `YOUR CODE HERE`, and that no tests fail.  

Note: on Colab, To use a GPU, do the following: Runtime$\rightarrow$Change runtime type$\rightarrow$ GPU

**Acknowledgements**: This notebook was writen with the help of the following
[turturial](https://towardsdatascience.com/understanding-topic-coherence-measures-4aa41339634c)


In [1]:
! pip install datasets
! pip install bertopic
! pip install --upgrade gensim


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 42.2 MB/s eta 0:00:00


# Loading 20_newsgroups dataset:

In [2]:
from datasets import load_dataset
# YOUR CODE HERE: fill which dataset (from HuggingFace hub) to use
hf_ds = "SetFit/20_newsgroups"
# END
newsgroups_dataset = load_dataset(hf_ds)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/734 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


train.jsonl:   0%|          | 0.00/14.8M [00:00<?, ?B/s]

test.jsonl:   0%|          | 0.00/8.91M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/11314 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7532 [00:00<?, ? examples/s]

In [3]:
newsgroups_dataset['test']

Dataset({
    features: ['text', 'label', 'label_text'],
    num_rows: 7532
})

In [4]:
# Lets look on example from the dataset:
next(iter(newsgroups_dataset['test']))

{'text': 'I am a little confused on all of the models of the 88-89 bonnevilles.\nI have heard of the LE SE LSE SSE SSEI. Could someone tell me the\ndifferences are far as features or performance. I am also curious to\nknow what the book value is for prefereably the 89 model. And how much\nless than book value can you usually get them for. In other words how\nmuch are they in demand this time of year. I have heard that the mid-spring\nearly summer is the best time to buy.',
 'label': 7,
 'label_text': 'rec.autos'}

## Loading BERTopic model

TODO: This part should be done by them, write some general insterction, and tell them to polt topic info, and to create a model_topics variable which contain the list of topics (list of list of strs). Also ask them what k (number of words in topic) the model that they use have.

In [11]:
from bertopic import BERTopic
# YOUR CODE HERE: fill model (from HuggingFace hub) to use
hf_model = "ctam8736/bertopic-20-newsgroups"
# END

topic_model = BERTopic.load(hf_model)
topic_info = topic_model.get_topic_info()
topic_info
# Note from BERTopic doc: Topic number -1 refers to all outliers and should typically be ignored. Next, let's take a look at the most frequent topic that was generated

topics.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

topic_embeddings.safetensors:   0%|          | 0.00/207k [00:00<?, ?B/s]

,Topic,Count,Name,Representation,Representative_Docs
0,-1,3737,-1_article_information_subject_re,"[article, information, subject, re, what, some...",NaN
1,0,976,0_scsi_scsi2_scsi1_drives,"[scsi, scsi2, scsi1, drives, bios, ram, disk, ...",NaN
2,1,918,1_nhl_puck_leafs_flyers,"[nhl, puck, leafs, flyers, pitching, hockey, r...",NaN
3,2,409,2_firearm_firearms_handgun_guns,"[firearm, firearms, handgun, guns, gun, handgu...",NaN
4,3,387,3_ford_honda_nissan_bmw,"[ford, honda, nissan, bmw, dealer, car, toyota...",NaN
...,...,...,...,...,...
130,129,11,129_waving_wave_waves_bikers,"[waving, wave, waves, bikers, bikes, biker, ha...",NaN
131,130,11,130_vocoder_compressionalgorithms_compression_...,"[vocoder, compressionalgorithms, compression, ...",NaN
132,131,10,131_mouse_jumpiness_mousecom_mouseits,"[mouse, jumpiness, mousecom, mouseits, jumps, ...",NaN
133,132,10,132_netware_lan_workgroup_workgroups,"[netware, lan, workgroup, workgroups, w4wg, ne...",NaN


In [12]:
model_topics = topic_info['Representation'].to_list()

# Evaluation using Gensim

In [13]:
import re
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora.dictionary import Dictionary
from gensim.corpora.dictionary import Dictionary

tokenizer = lambda s: re.findall( '[\w\d]+', s.lower() )
texts = [tokenizer(s['text']) for s in newsgroups_dataset['test']]

word2id = Dictionary(texts)


<>:6: SyntaxWarning: invalid escape sequence '\w'
<>:6: SyntaxWarning: invalid escape sequence '\w'
/tmp/ipython-input-3317336189.py:6: SyntaxWarning: invalid escape sequence '\w'
  tokenizer = lambda s: re.findall( '[\w\d]+', s.lower() )


In [14]:
# YOUR CODE HERE: fill which metric to use (supported according to CoherenceModel documentation):
coherence_metrice_1 = 'c_v'
coherence_metrice_2 = 'c_npmi'

# END

In [15]:
cm_1 = CoherenceModel(topics = model_topics, texts=texts,  dictionary=word2id, coherence=coherence_metrice_1)
print(f"{coherence_metrice_1}: {round(cm_1.get_coherence(), 2)}")
cm_2 = CoherenceModel(topics = model_topics, texts=texts,  dictionary=word2id, coherence=coherence_metrice_2)
print(f"{coherence_metrice_2}: {round(cm_2.get_coherence(), 2)}")


c_v: 0.41
c_npmi: -0.17
